# install simple salesforce

In [ ]:
#pip install simple_salesforce

In [ ]:
#Prerequisites

import sys
!{sys.executable} -m pip install --user networkx
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user pandas

import networkx as nx
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import requests
import csv
import glob
import scipy.sparse

from simple_salesforce import Salesforce
from io import StringIO
from pandas import DataFrame

In [ ]:
#sf = Salesforce(username='',password='', security_token='')

sf_instance = 'https://ua-trellis.lightning.force.com/' #Your Salesforce Instance URL
reportId = '00O6R000008QxpyUAC' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df1 = pd.read_csv(StringIO(download_report))

In [ ]:
os.getcwd()

directory = '/Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/'

for file_name in glob.glob(directory+'*.csv'):
    x = np.genfromtxt(file_name,delimiter=',')[:,2]
    # do your calculations


# Define the directory path
as /Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce
directory = '/Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/'

# Get a list of all files in the directory
files = os.listdir(directory)

# Filter out only the CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Loop through each CSV file and read it
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    # Now you can work with the DataFrame 'df' for each CSV file
    print("Loaded:", file)



In [ ]:
# Read in Source File - NB this must match the schema requirements
# read in data
# /Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce
cases_report_SF = pd.read_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/POC_Network_Analysis_Cases.csv')
initial_report_SF = pd.read_csv("~/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/practice_appointments_POC.csv")
#CodeType = 'latin1' # https://docs.python.org/3/library/codecs.html#standard-encodings
print(initial_report_SF.tail())


In [ ]:
print(cases_report_SF.tail())

In [ ]:
print(initial_report_SF.head())

In [ ]:
# Filter out rows where case record type and department is NA or empty
cases_report_SF = cases_report_SF.dropna(subset=['Case Record Type', 'Department'])
cases_report_SF.shape

In [ ]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
initial_report_SF = initial_report_SF.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
initial_report_SF.shape

In [ ]:
# trim down the cases to make the merge possible
initial_report_SF = initial_report_SF[
    (~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")
]

initial_report_SF = initial_report_SF[
    (~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")
]

cases_report_SF = cases_report_SF[
    (~cases_report_SF['Case Record Type'].isna()) | (cases_report_SF['Case Record Type'] != "")
]

cases_report_SF = cases_report_SF[
    (~cases_report_SF['Department'].isna()) | (cases_report_SF['Department'] != "")
]


In [ ]:
# Merge the DataFrames
filtered_df = pd.merge(cases_report_SF, initial_report_SF, how='left', left_on='Emplid', right_on='Contact EmplID')


In [ ]:
filtered_df.shape

In [ ]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
#filtered_df = merged_df.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
#initial_report_SF = initial_report_SF[(~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")]
#initial_report_SF = initial_report_SF[(~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")]
#print(filtered_df.head())

In [ ]:
# Convert 'Appointment Date' column to datetime
filtered_df['AppointmentDate'] = pd.to_datetime(filtered_df['Appointment Date'])

# Calculate today's date
today = pd.to_datetime('today').normalize()
filtered_df['today'] = today

# Calculate days since 'AppointmentDate'
filtered_df['days_since'] = (filtered_df['today'] - filtered_df['AppointmentDate']).dt.days

# filter dates to 2024
#filtered_df[(filtered_df['date']>datetime.date(2024,1,1)) & (filtered_df['date']<datetime.date(2024,5,5))]
filtered_df = filtered_df[(filtered_df['AppointmentDate'] > "2024-01-01") & (filtered_df['AppointmentDate'] < "2024-07-01")]
# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
# create new emplID's
# Extract unique IDs
unique_ids = filtered_df['Contact EmplID'].unique()

# Generate new IDs (random IDs)
np.random.seed(53013)  # for reproducibility
new_ids = np.random.randint(1000, 9999, len(unique_ids))

# Create a dictionary to map old IDs to new IDs
id_mapping = dict(zip(unique_ids, new_ids))

id_mapping
#filtered_df_newID=filtered_df.append(id_mapping,ignore_index=True)


In [ ]:
#filtered_df['Contact EmplID'].map(id_mapping)  
filtered_df['New_ID'] = filtered_df['Contact EmplID'].map(id_mapping)

In [ ]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

In [ ]:
# create a new column to extract the names
# Extract just the names
filtered_df['Student Name'] = filtered_df['Appointment Name'].str.split(' - ', expand=True)[1]

# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

In [ ]:
# add an index column

filtered_df['index'] = range(1, len(filtered_df) + 1)
#filtered_df['index'] = filtered_df.index

# Display the updated DataFrame
#print(filtered_df.head())

In [ ]:
# create a source column
filtered_df['Source'] = filtered_df.groupby(['Advisor Name']).ngroup()

# Display the updated DataFrame
print(filtered_df.head())

In [ ]:
filtered_df = filtered_df.rename(columns={'Owner: Owner ID': 'owner_id', 'New_ID': 'Target'})

In [ ]:
filtered_df.info()

In [ ]:
filtered_df['Source'] = filtered_df['Source'].map(str)
filtered_df['Target'] = filtered_df['Target'].map(str)

In [ ]:
# Add directions to d3
filtered_df['Direction'] = filtered_df['Source'] + ' -> ' + filtered_df['Target']

In [ ]:
filtered_df.info()

In [ ]:
# Group by 'NAME' and check if any row within the group has a non-null 'DEPARTMENT'
grouped = filtered_df.groupby('Advisor Name')['Department'].transform('first')

In [ ]:
# Impute the 'DEPARTMENT' value within each group
filtered_df['Department'] = filtered_df['Department'].fillna(grouped)

In [ ]:
# FOLLOWING KNIPPENBERG

# Save the DataFrame to a CSV file

filtered_df.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/py-filtered_merged_df.csv', index=False)  # Set index=False to exclude the index from the CSV file


In [ ]:
#reduced_df = filtered_df.sample(n=300)

In [ ]:
arr_SrcTgt = np.array(filtered_df[['Source','Target']])

In [ ]:
# Create Network Graph Coordinates...
Q = nx.MultiDiGraph()
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q) 


In [ ]:
# Create Graph Coordinates File...
df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/CoordsFile_merged.csv', 
                     index_label='NodeName')
#print(df_Raw_Coords)

In [ ]:
# Create Bridge File... 
# Tableau Code: IF [Src-Tgt]/2 = ROUND([Src-Tgt]/2) THEN 'Source' ELSE 'Target' END
arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2)) 
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1) 
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName']) 
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
    arr_Index.append(i)
    arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index 

df_SrcTgt.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/BridgeFile_merged.csv',
                 index_label='Src-Tgt')
#print(df_ScrTgt.head())

In [ ]:
nx.draw(dict_Coords)
plt.show()

In [ ]:
#nx.draw(Q, pos=nx.spring_layout(Q));
nx.draw(Q, pos=nx.spring_layout(Q), with_labels=True, node_size=100, node_color="skyblue", edge_color="gray")
plt.show()

In [ ]:
# betweenness centrality
plt.hist(nx.centrality.closeness_centrality(Q).values());

In [ ]:
# nx.diameter(Q)

In [ ]:
# nx.cluster.average_clustering(Q)

In [ ]:
# degree distribution (a histogram of how many edges each node has)

plt.hist([v for k,v in nx.degree(Q)]);

In [ ]:

degrees = [v for k, v in nx.degree(Q)]

# Calculate the range of your data
min_degree = min(degrees)
max_degree = max(degrees)

# Calculate the number of bins based on increments of 10
num_bins = int((max_degree - min_degree) / 10) + 1

# Plot the histogram
plt.hist(degrees, bins=num_bins)
plt.xlim(0, 200)
plt.show()

In [ ]:
print('Run Completed Successfully')

In [ ]:
degrees = [v for k, v in nx.degree(Q)]

# Calculate the range of your data
min_degree = min(degrees)
max_degree = max(degrees)  # Truncate the maximum degree to 600

# Calculate the number of bins based on increments of 5
num_bins = int((max_degree - min_degree) / 5) + 1

# Plot the histogram
plt.hist(degrees, bins=num_bins)
plt.xlim(0, 200)  # Set the limit of x-axis from 0 to 600
plt.show()